<a href="https://colab.research.google.com/github/aneeq-shaffy/DL-labsheets/blob/main/paddy_rice_disease_detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install kagglehub
import kagglehub

dataset_path = kagglehub.dataset_download(
    "tntiphan/paddy-rice-disease-classification"
)

print(dataset_path)

Using Colab cache for faster access to the 'paddy-rice-disease-classification' dataset.
/kaggle/input/paddy-rice-disease-classification


In [2]:
import json
import os

id2label_path = os.path.join(dataset_path, 'id2label.json')
with open(id2label_path, 'r') as f:
    id2label = json.load(f)

print("id2label.json content:", id2label)

# Also read label2id.json for completeness, as they usually come in pairs
label2id_path = os.path.join(dataset_path, 'label2id.json')
with open(label2id_path, 'r') as f:
    label2id = json.load(f)

print("label2id.json content:", label2id)

id2label.json content: {'0': 'bacterial_leaf_blight', '1': 'brown_spot', '2': 'healthy', '3': 'leaf_blast'}
label2id.json content: {'bacterial_leaf_blight': 0, 'brown_spot': 1, 'healthy': 2, 'leaf_blast': 3}


In [3]:
import pandas as pd
import os

metadata_path = os.path.join(dataset_path, 'metadata.csv')
metadata_df = pd.read_csv(metadata_path)
display(metadata_df.head())

,path,label,split
0,/kaggle/input/paddy-rice-disease-classificatio...,leaf_blast,train
1,/kaggle/input/paddy-rice-disease-classificatio...,brown_spot,train
2,/kaggle/input/paddy-rice-disease-classificatio...,bacterial_leaf_blight,train
3,/kaggle/input/paddy-rice-disease-classificatio...,bacterial_leaf_blight,train
4,/kaggle/input/paddy-rice-disease-classificatio...,brown_spot,train


In [4]:

NUM_CLASSES = 4
IMG_SIZE = (224, 224)
BATCH_SIZE = 32


In [5]:
import tensorflow as tf

base_model = tf.keras.applications.EfficientNetB0(
    include_top=False,
    weights="imagenet",
    input_shape=(224, 224, 3)
)

base_model.trainable = False


16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


In [6]:
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(NUM_CLASSES, activation="softmax")
])


In [7]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)


In [8]:
!pip install kagglehub
import kagglehub

dataset_path = kagglehub.dataset_download(
    "tntiphan/paddy-rice-disease-classification"
)

print(dataset_path)

Using Colab cache for faster access to the 'paddy-rice-disease-classification' dataset.
/kaggle/input/paddy-rice-disease-classification


In [9]:
print(dataset_path)

/kaggle/input/paddy-rice-disease-classification


In [10]:
import os

os.listdir("/kaggle/input/paddy-rice-disease-classification")


['label2id.json', 'metadata.csv', 'data', 'id2label.json']

In [11]:
import os

os.listdir("/kaggle/input/paddy-rice-disease-classification/data")


['bacterial_leaf_blight', 'brown_spot', 'healthy', 'leaf_blast']

In [12]:
NUM_CLASSES = 4


In [13]:
import tensorflow as tf

base_model = tf.keras.applications.EfficientNetB0(
    include_top=False,
    weights="imagenet",
    input_shape=(224, 224, 3)
)

base_model.trainable = False


In [14]:
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(NUM_CLASSES, activation="softmax")
])


In [15]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)


In [16]:
import tensorflow as tf

DATASET_PATH = "/kaggle/input/paddy-rice-disease-classification/data"
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

train_ds = tf.keras.utils.image_dataset_from_directory(
    DATASET_PATH,
    validation_split=0.2,
    subset="training",
    seed=42,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    DATASET_PATH,
    validation_split=0.2,
    subset="validation",
    seed=42,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

# EfficientNet preprocessing
from tensorflow.keras.applications.efficientnet import preprocess_input

train_ds = train_ds.map(lambda x, y: (preprocess_input(x), y))
val_ds   = val_ds.map(lambda x, y: (preprocess_input(x), y))

# Optional performance boost
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.prefetch(AUTOTUNE)
val_ds   = val_ds.prefetch(AUTOTUNE)


Found 15291 files belonging to 4 classes.
Using 12233 files for training.
Found 15291 files belonging to 4 classes.
Using 3058 files for validation.


In [17]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10
)


Epoch 1/10
383/383 ━━━━━━━━━━━━━━━━━━━━ 256s 594ms/step - accuracy: 0.6498 - loss: 0.8839 - val_accuracy: 0.8466 - val_loss: 0.4480
Epoch 2/10
383/383 ━━━━━━━━━━━━━━━━━━━━ 135s 351ms/step - accuracy: 0.8382 - loss: 0.4384 - val_accuracy: 0.8748 - val_loss: 0.3611
Epoch 3/10
383/383 ━━━━━━━━━━━━━━━━━━━━ 115s 301ms/step - accuracy: 0.8578 - loss: 0.3851 - val_accuracy: 0.8875 - val_loss: 0.3270
Epoch 4/10
383/383 ━━━━━━━━━━━━━━━━━━━━ 142s 300ms/step - accuracy: 0.8737 - loss: 0.3528 - val_accuracy: 0.8967 - val_loss: 0.3008
Epoch 5/10
383/383 ━━━━━━━━━━━━━━━━━━━━ 134s 350ms/step - accuracy: 0.8810 - loss: 0.3320 - val_accuracy: 0.8937 - val_loss: 0.2960
Epoch 6/10
383/383 ━━━━━━━━━━━━━━━━━━━━ 115s 301ms/step - accuracy: 0.8875 - loss: 0.3144 - val_accuracy: 0.9022 - val_loss: 0.2882
Epoch 7/10
383/383 ━━━━━━━━━━━━━━━━━━━━ 135s 353ms/step - accuracy: 0.8902 - loss: 0.2982 - val_accuracy: 0.9012 - val_loss: 0.2836
Epoch 8/10
383/383 ━━━━━━━━━━━━━━━━━━━━ 124s 305ms/step - accuracy: 0.8910 -

In [18]:
val_loss, val_acc = model.evaluate(val_ds)
print(f"Validation accuracy: {val_acc*100:.2f}%")


96/96 ━━━━━━━━━━━━━━━━━━━━ 23s 242ms/step - accuracy: 0.8978 - loss: 0.3153
Validation accuracy: 90.65%


In [19]:
model.save("/kaggle/working/rice_disease_effnet.h5")


In [20]:
import tensorflow as tf

model = tf.keras.models.load_model("/kaggle/working/rice_disease_effnet.h5")


In [21]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

with open("/kaggle/working/rice_disease_effnet.tflite", "wb") as f:
    f.write(tflite_model)


Saved artifact at '/tmp/tmp5i1yxnc3'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='input_layer_3')
Output Type:
  TensorSpec(shape=(None, 4), dtype=tf.float32, name=None)
Captures:
  136597054956048: TensorSpec(shape=(1, 1, 1, 3), dtype=tf.float32, name=None)
  136597054960656: TensorSpec(shape=(1, 1, 1, 3), dtype=tf.float32, name=None)
  136597054961040: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136597054956624: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136597054957008: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136597054960848: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136597054960272: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136597054959696: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136597054958544: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136597054961616: TensorSpec(shape=(), dtype=tf.resource, name=Non

In [22]:
import os

print(os.path.exists("/kaggle/working/rice_disease_effnet.tflite"))



True


In [23]:
import os

size_mb = os.path.getsize("/kaggle/working/rice_disease_effnet.tflite") / (1024 * 1024)
print(f"{size_mb:.2f} MB")


4.34 MB
